In [ ]:
import shutil
import requests
from urllib.parse import urlparse
import sys
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from langchain_core.language_models import BaseChatModel
import json
import datasets
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import HuggingFaceHub
from langchain_community.chat_models import ChatHuggingFace
import os
import random
import time
from datasets import Dataset, DatasetDict
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import config 

In [4]:
pdfPath = config.pdfPath
model_id = config.model_id

In [5]:
if pdfPath is None:
    raise ValueError("pdfPath is None. Please set the  pdf path in config.py.")

if model_id is None:
    raise ValueError("model_id is None. Please set the model_id in config.py.")

In [6]:
loader = PyPDFLoader(pdfPath)

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,  
        chunk_overlap=200,
        add_start_index=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )
try:
    langchain_docs = loader.load_and_split(text_splitter=text_splitter) #loads and slits
    #docs = loader.load()
    #langchain_docs = text_splitter.split_documents(docs)
except Exception as e:
    print("An error occurred:", e)

In [8]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

db = FAISS.from_documents(langchain_docs,
                          HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5'))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
d_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map=d_map)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
import re
prompt_template = """
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Instruction:
  {query}
  
  ### Answer:
  """

prompt_template_with_context = """
  Below is an instruction that describes a task, paired with an input that provides further context. Write a response that 
  appropriately completes the request.
  Each response should at least 30 words. please end the answer with fullstop(.)
  ### Instruction:
  {query}
  ### Input:
  {input}
  
  ### Answer:
  """



def parse_answer_from_prompt(prompt_template_with_context):
    answer_start = prompt_template_with_context.find("### Answer:")
    if answer_start == -1:
        return "Answer not found."
    return prompt_template_with_context[answer_start + len("### Answer:"):].strip()



def remove_special_characters_and_after(input_string):
    # Define a regular expression pattern to match special characters excluding . , and -
    pattern = r'[^a-zA-Z0-9\s\t.,-]'
    
    # Use re.split() to split the string based on the pattern
    parts = re.split(pattern, input_string)
    
    # The first part will contain the portion of the string without special characters
    cleaned_string = parts[0]
    
    return cleaned_string

#Inference Function............
def inferance(prompt: str, model, tokenizer, temp = 1.0, limit = 400, input = False) -> str:
  device = "cuda:0"
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(**model_inputs, max_new_tokens=limit, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)

  answer = decoded[0].split("### Answer:")[1]
  pattern = r"[^\w\s,.'\)\"]" 
  split_string = re.split(pattern, answer)  
  # answer  = parse_answer_from_prompt(decoded[0])
  # answer  = remove_special_characters(answer) 
  # print(decoded[0])  
  return split_string[0]

In [12]:
generated_questions = pd.read_csv("datasets/db2_dataset.csv")
datapoints = Dataset.from_pandas(generated_questions)

In [ ]:
outputs = []
i = 1
n = len(datapoints)
for datapoint in datapoints:
    print("Inference number {}/{} in progress.....".format(i, n))
    if i >= 15 and i <= 250:
        question = datapoint["question"]
        correctAnswer = datapoint["answer"]
        relevantText = db.as_retriever(search_type="mmr").get_relevant_documents(question)
        context = relevantText[0].page_content
        prompt = prompt_template_with_context.format(query=question, input =context)
        answer = inferance(prompt, model, tokenizer, temp = 0.1, limit = 400, input = False)
        outputs.append(
                {   
                    "question": question,
                    "correctAnswer": correctAnswer,
                    "ragAnswer": answer,
                
                }
            )
        generated_questions = pd.DataFrame.from_dict(outputs)
        generated_questions.to_csv("eval.csv", index=False)
    i += 1
    

Inference number 1/263 in progress.....
Inference number 2/263 in progress.....
Inference number 3/263 in progress.....
Inference number 4/263 in progress.....
Inference number 5/263 in progress.....
Inference number 6/263 in progress.....
Inference number 7/263 in progress.....
Inference number 8/263 in progress.....
Inference number 9/263 in progress.....
Inference number 10/263 in progress.....
Inference number 11/263 in progress.....
Inference number 12/263 in progress.....
Inference number 13/263 in progress.....
Inference number 14/263 in progress.....
Inference number 15/263 in progress.....
Inference number 16/263 in progress.....
Inference number 17/263 in progress.....
Inference number 18/263 in progress.....
Inference number 19/263 in progress.....
Inference number 20/263 in progress.....
Inference number 21/263 in progress.....
Inference number 22/263 in progress.....
Inference number 23/263 in progress.....
Inference number 24/263 in progress.....
Inference number 25/263 i